In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import json

<h5>Cleaning up Yahoo data</h5>

In [2]:
def clean_yahoo_df(path):
    ticker = path.split('/')[-1].split('.')[0]
    df = pd.read_csv(path)
    
    df.drop(['High','Low','Open','Close'], axis=1, inplace=True)
    df['Adj Close'] = df['Adj Close'].round(2)
    #df['Volume'] = df['Volume'].astype('int32')
    df.rename(columns={'Volume': f'{ticker}_volume', 'Adj Close': ticker}, inplace=True)
    df.set_index('Date', inplace=True)
    
    return df

In [3]:
main_df = clean_yahoo_df('final_dfs/spy.csv')
main_df.head()

,spy_volume,spy
Date,,
1993-01-29,1003200.0,26.30
1993-02-01,480500.0,26.49
1993-02-02,201300.0,26.54
1993-02-03,529400.0,26.82
1993-02-04,531500.0,26.94


A few economic indicators are updated weekly, monthly, etc. Some of those days are on the weekend when equity markets are not opened. For that, we will have to outer join those dataframes with the main dataframes, forward fill the NaN data, and remove weekend dates. Below is the list of SPY dates that will be used later.

In [4]:
spy_dates = list(main_df.index)

Below are all tickers pulled from Yahoo and saved locally. I will need to find a way to automate the refresh daily but for now will use the data I have to optimize the model.

Also note, for the individual sector ETFs, I will use Vanguard ETFs instead of SPDR even if SPY is an SPDR ETFs. Even though most of the SPDR ETFs are larger and more liquid I find the Vanguard ETFs more accurately reflects their respective idustries as the portfolios include small and mid caps. SPDR ETFs are heavily concentrated in stocks already included in SPY.

In [5]:
# note: leaving out USO becuase we have /CL

etfs = ['qqq', 'iwm']

sector_etf_path = 'sector etfs - Vanguard'
sector_etf = ['vaw', 'vcr', 'vdc', 'vde', 'vfh', 'vgt', 'vht', 'vis', 'vnq', 'vox', 'vpu']

futures_path = 'futures'
futures = ['cl=f', 'gc=f']

bonds_etf_path = 'bonds etfs'
bonds = ['ief', 'tlt']

In [6]:
for ticker in etfs:
    temp_df = clean_yahoo_df(f'final_dfs/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

for ticker in sector_etf:
    temp_df = clean_yahoo_df(f'final_dfs/{sector_etf_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

for ticker in futures:
    temp_df = clean_yahoo_df(f'final_dfs/{futures_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')
    
for ticker in bonds:
    temp_df = clean_yahoo_df(f'final_dfs/{bonds_etf_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

In [7]:
main_df.head()

,spy_volume,spy,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,...,vpu_volume,vpu,cl=f_volume,cl=f,gc=f_volume,gc=f,ief_volume,ief,tlt_volume,tlt
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,1003200.0,26.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,480500.0,26.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,201300.0,26.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,529400.0,26.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,531500.0,26.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h5>Other charts (Treasure bond yields, Libor rate, Mortgage rate, M1 & M2 money supply, VIX, DIX)</h5>
<br>
Data pulled from various sources.

In [8]:
# maybe include VVIX in the future?
# Notes:
# 30 year fixed mortgage rate is updated monthly (need may numbers)
# money supply data is updated weekly

economics_path = 'economics'
economics = ['CPIAUCSL', 'DFF', 'DGS10', 'DGS30', 'M1', 'M2', 'MORTGAGE15US', 'MORTGAGE30US',
             'T10YIE', 'UNRATE', 'USD1MTD156N', 'USD6MTD156N', 'USD12MD156N']
economic_names = ['CPI', 'fed_funds_rate', '10_yr_treasury', '30_yr_treasury', 'M1_supply',
                  'M2_supply', '15_yr_mortgage', '30_yr_mortgage', '10_yr_inflation',
                  'unemployment', '1m_libor', '3m_libor', '12m_libor']

currency_path = 'currency'
currency = ['dxy']

vix = '^vix'
dix = 'DIX'

In [9]:
# economic data
for code, name in zip(economics, economic_names):
    try:
        economics_df = pd.read_csv(f'final_dfs/{economics_path}/{code}.csv')
        economics_df = economics_df.loc[economics_df['DATE'] >= '1993-01-01']
        economics_df.rename(columns={code: name}, inplace=True)
        economics_df.set_index('DATE', inplace=True)
        if code in ['M1_supply', 'M2_supply']:
            economics_df = economics_df * 1000000000
        elif code == 'GFDEBTN':
            economics_df = economics_df * 1000000
        elif code == 'CPIAUCSL':
            economics_df
        else:
            economics_df = economics_df / 100
    except:
        economics_df.replace('.', np.nan, inplace=True)
        economics_df.ffill(inplace=True)
    main_df = main_df.join(economics_df, how='outer')

# currency (check date format)
currency_df = pd.read_csv(f'final_dfs/{currency_path}/{currency[0]}.csv')
currency_df.drop([' Open',' High',' Low'], axis=1, inplace=True)
currency_df.rename(columns={' Close': currency[0]}, inplace=True)
currency_df['Date'] = currency_df['Date'].apply(lambda x: dt.strptime(x, '%m/%d/%y').strftime('%Y-%m-%d'))
currency_df.set_index('Date', inplace=True)
main_df = main_df.join(currency_df, how='left')

# vix
vix_df = clean_yahoo_df(f'final_dfs/{vix}.csv')
vix_df.drop('^vix_volume', axis=1, inplace=True)
main_df = main_df.join(vix_df, how='left')

# dix
dix_df = pd.read_csv(f'final_dfs/{dix}.csv')
dix_df.drop('price', axis=1, inplace=True)
dix_df.set_index('date', inplace=True)
main_df = main_df.join(dix_df, how='left')

In [10]:
main_df.head(3)

,spy_volume,spy,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,...,30_yr_mortgage,10_yr_inflation,unemployment,1m_libor,3m_libor,12m_libor,dxy,^vix,dix,gex
1993-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.073,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Forward fill any data that is updated weekly, monthly, etc. Remove all dates when market is closed (weekends, holidays).

In [11]:
main_df.ffill(inplace=True)

In [12]:
main_df = main_df[main_df.index.isin(spy_dates)]
main_df.head()

,spy_volume,spy,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,...,30_yr_mortgage,10_yr_inflation,unemployment,1m_libor,3m_libor,12m_libor,dxy,^vix,dix,gex
1993-01-29,1003200.0,26.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0786,NaN,0.073,3.12500,3.36719,3.68750,92.48,12.42,NaN,NaN
1993-02-01,480500.0,26.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0786,NaN,0.071,3.12500,3.37500,3.68750,93.39,12.33,NaN,NaN
1993-02-02,201300.0,26.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0786,NaN,0.071,3.18750,3.43750,3.75000,93.94,12.25,NaN,NaN
1993-02-03,529400.0,26.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0786,NaN,0.071,3.18750,3.43750,3.75000,94.33,12.12,NaN,NaN
1993-02-04,531500.0,26.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0786,NaN,0.071,3.18750,3.43750,3.75000,94.33,12.29,NaN,NaN


<h5>Handling Null Values</h5>
<br>
I have decided to drop any rows with NaN. DIX/GEX data had the most NaN values since it was not recorded until 2011. I could just remove the columns but the main purpose of this project was to see how accurate the DIX index was at predicting SPY prices so I left it in.
<br>
Will do another analysis without DIX/GEX. When that happens we will have data from ~2004

In [13]:
main_df.isna().sum().sort_values(ascending=False).head(10)

gex           4598
dix           4598
vox           2939
vox_volume    2939
vnq           2939
vnq_volume    2939
vde_volume    2939
vde           2939
vis           2939
vis_volume    2939
dtype: int64

In [14]:
main_df = main_df.dropna(axis=0)
main_df.head()

,spy_volume,spy,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,...,30_yr_mortgage,10_yr_inflation,unemployment,1m_libor,3m_libor,12m_libor,dxy,^vix,dix,gex
2011-05-02,126278700.0,113.55,48149400.0,53.57,48962400.0,74.88,59000.0,74.33,119000.0,58.26,...,0.0478,2.56,0.09,0.21025,0.43050,0.76100,72.95,15.99,0.378842,1.897313e+09
2011-05-03,138375000.0,113.14,38100000.0,53.31,56070000.0,73.94,65800.0,73.38,40400.0,58.00,...,0.0478,2.55,0.09,0.20950,0.43025,0.75775,73.14,16.70,0.383411,1.859731e+09
2011-05-04,182678500.0,112.39,53097200.0,53.23,64754600.0,72.98,89000.0,72.09,26000.0,57.78,...,0.0478,2.55,0.09,0.20900,0.42850,0.75625,73.03,17.08,0.392122,1.717764e+09
2011-05-05,226900000.0,111.37,72200000.0,52.94,91858900.0,72.65,89400.0,71.42,26300.0,57.68,...,0.0471,2.48,0.09,0.20620,0.42700,0.75200,74.19,18.20,0.405457,1.361864e+09
2011-05-06,222787200.0,111.86,72507000.0,53.11,76446200.0,73.03,101100.0,71.95,23800.0,57.80,...,0.0471,2.48,0.09,0.20385,0.42500,0.74900,74.84,18.40,0.418649,1.490329e+09


In [15]:
spy_price = main_df['spy']
tickers_df = main_df.drop('spy', axis=1)
tickers_column = tickers_df.columns

In [ ]:
#specify columns to scale instead of the entire dataset (see if this is worth it)

In [16]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
tickers_df = pd.DataFrame(scalar.fit_transform(tickers_df), columns=tickers_column)
#tickers_df = scalar.transform(tickers_df)
tickers_df.head()

,spy_volume,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,vdc_volume,...,30_yr_mortgage,10_yr_inflation,unemployment,1m_libor,3m_libor,12m_libor,dxy,^vix,dix,gex
0,0.151971,0.198375,0.042005,0.235542,0.182481,0.020270,0.256841,0.061954,0.067085,0.026170,...,0.906433,0.962617,0.491071,0.026320,0.042921,0.087144,0.000000,0.093134,0.263520,0.334306
1,0.169312,0.149835,0.040643,0.269736,0.174321,0.022890,0.245826,0.017685,0.065421,0.028992,...,0.906433,0.957944,0.491071,0.026004,0.042824,0.085899,0.006271,0.102787,0.288455,0.331719
2,0.232824,0.222274,0.040224,0.311515,0.165987,0.031830,0.230867,0.009575,0.064012,0.036975,...,0.906433,0.957944,0.491071,0.025793,0.042148,0.085324,0.002640,0.107954,0.335992,0.321944
3,0.296219,0.314543,0.038705,0.441909,0.163122,0.031985,0.223098,0.009744,0.063372,0.023469,...,0.865497,0.925234,0.491071,0.024614,0.041569,0.083696,0.040924,0.123182,0.408768,0.297442
4,0.290323,0.316026,0.039596,0.367761,0.166421,0.036493,0.229244,0.008336,0.064140,0.016146,...,0.865497,0.925234,0.491071,0.023625,0.040796,0.082547,0.062376,0.125901,0.480759,0.306286


In [ ]:
# CPI needs April data
# MI & M2 missing last week of April data
# explore adding options data if possible
# normalize numbers
# experiment with a few models